# Spark JDBC
+ MariaDB나 Oracle에 저장된 테이블을 불러와서
+ 분석을 하려면 pyspark 실행시 JDBC 드라이버를 지정해야 함
+ pyspark --driver-class-path '사용할 jdbc 드라이버'

In [1]:
# 디비버 접속 유저, 패스워드

url = 'jdbc:mysql://bigdata.cjsnjphl4oxy.ap-northeast-2.rds.amazonaws.com:3306/bigdata'
user = 'admin'
password = 'bigdata2020'

In [2]:
emp = spark.read.format('jdbc').options(url=url, user=user, password=password, dbtable='employees').load()

In [3]:
emp.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: double (nullable = true)
 |-- MANAGER_ID: integer (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [4]:
emp.select('FIRST_NAME').show(5)

+----------+
|FIRST_NAME|
+----------+
|    Steven|
|     Neena|
|       Lex|
| Alexander|
|     Bruce|
+----------+
only showing top 5 rows



In [5]:
EMP = emp.createOrReplaceTempView('EMP')

In [6]:
spark.sql('select FIRST_NAME from EMP').show(5)

+----------+
|FIRST_NAME|
+----------+
|    Steven|
|     Neena|
|       Lex|
| Alexander|
|     Bruce|
+----------+
only showing top 5 rows



21/10/11 05:49:56 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [11]:
# summermedals.csv 파일을 mariadb 서버에 업로드하고
# spark를 이용해서 금메달을 가장 많이 딴 나라순으로 조회해보세요
# 단, 조회방식은 spark DSL과 SQL을 이용합니다

summermedals = spark.read.format('jdbc').options(url=url, user=user, password=password, dbtable='summermedals').load()

In [12]:
summermedals.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Athlete: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Event: string (nullable = true)
 |-- Medal: string (nullable = true)



In [13]:
SUMMERMEDALS = summermedals\
         .createOrReplaceTempView('SUMMERMEDALS')

In [14]:
sql = """ select Country, count(Medal) cnt 
   from SUMMERMEDALS where Medal = 'Gold' 
   group by Country order by cnt desc """
spark.sql(sql).show(5)

[Stage 3:==================================================>    (182 + 1) / 200]

+-------+----+
|Country| cnt|
+-------+----+
|    USA|1740|
|    URS| 760|
|    GBR| 454|
|    ITA| 339|
|    GER| 330|
+-------+----+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import count

In [ ]:
summermedals.filter(col('Medal') == 'Gold')\
  .groupBy('Country')\
  .agg(count(col('Medal')).alias('cnt'))\
  .orderBy('cnt', ascending=False)\
.show(10)